<a href="https://colab.research.google.com/github/Barleysack/DaconRepo/blob/main/Dacon_Bosung_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
!pip install transformers

Mounted at /content/gdrive/
     |████████████████████████████████| 3.5 MB 8.1 MB/s 
     |████████████████████████████████| 6.8 MB 66.4 MB/s 
     |████████████████████████████████| 596 kB 68.4 MB/s 
     |████████████████████████████████| 895 kB 93.0 MB/s 
     |████████████████████████████████| 67 kB 7.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
from glob import glob
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
from tqdm import tqdm, tqdm_notebook
import re
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from transformers import pipeline, AutoTokenizer ,AutoModelForSequenceClassification,AutoConfig,Trainer,TrainingArguments , AutoModel
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import AdamW

In [3]:
data_path = '/content/gdrive/MyDrive/Data'
train= pd.read_csv(os.path.join(data_path,'train_data.csv'))
test = pd.read_csv(os.path.join(data_path,'test_data.csv'))


In [4]:
train.head()

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral


In [5]:
premise_len_mean = np.mean(train['premise'].str.len()) # 리뷰 길이의 평균값 계산
print('실제 Text 평균 길이 :',premise_len_mean)

hypo_len_mean = np.mean(train['hypothesis'].str.len()) # 리뷰 길이의 평균값 계산
print('가설 Text 평균 길이 :',hypo_len_mean)

max_prem_len = np.max(train['premise'].str.len())
print('최대 실제 텍스트 길이 : ', max_prem_len)

max_hypo_len = np.max(train['hypothesis'].str.len())
print('최대 가설 텍스트 길이 : ', max_hypo_len)

실제 Text 평균 길이 : 45.406552524201935
가설 Text 평균 길이 : 24.924433954716378
최대 실제 텍스트 길이 :  90
최대 가설 텍스트 길이 :  103


In [6]:
max_seq_len = 120
batch_size = 128
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate = 1e-5
model_name = "Huffon/klue-roberta-base-nli"
device = torch.device("cuda:0")
## 베이스라인 설정. 최대 길이만 변경해뒀습니다

In [7]:
class TorchDataset(Dataset):
  def __init__(self,premise,hypothesis,labels,tokenizer):
    premise = self.clean_text(premise)
    hypothesis = self.clean_text(hypothesis)
    self.len=len(premise)
    label2idx={key:value for value,key in enumerate(self.get_label_list())}
    self.labels=[label2idx[label] for label in labels]
    self.encodings = tokenizer(premise,hypothesis, add_special_tokens = True , return_tensors = 'pt', padding = True, truncation = True, max_length = max_seq_len)

  def __len__(self):
    return self.len
    
  def __getitem__(self,idx):
    item = {key: val[idx] for key,val in self.encodings.items()} 
    item['labels'] = self.labels[idx]
    return item

  def clean_text(self,series):
    return [self.clean(sent) for sent in series]

  def clean(self,sent):
    sent = re.sub('\u00A0',' ',sent)
    sent = re.sub(' +',' ',sent)
    return sent

  def get_data(self):
    return self.labels
  
  def get_label_list(self):
    return ['entailment','neutral','contradiction']



In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
train_dataset = TorchDataset(train['premise'],train['hypothesis'],train['label'],tokenizer)
model_config = AutoConfig.from_pretrained(model_name)
model_config.num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_name , config=model_config)

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/926 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

In [9]:
model.to('cuda:0')
print("model preparation done")

model preparation done


In [10]:
train_dataloader = DataLoader(
    dataset = train_dataset,
    pin_memory = (torch.cuda.is_available()),
    shuffle=True,
    batch_size=batch_size,
    num_workers = 4,
    drop_last = True
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer,
                                                     T_max=10,
                                                     eta_min=0.001)

In [11]:
def train_loop(model,num_epochs,train_dataloader):
    for epoch in range(num_epochs):

        run_count = 0
        avg_loss = 0

        running_loss, correct, total = 0.0, 0,0
        preds, gt = [], []
        pbar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
        model.train()

        for batch, data in pbar:
            
            labels = data['labels'].to("cuda:0")
            data['attention_mask'] = data['attention_mask'].to(device)
            data['input_ids'] = data['input_ids'].to(device)
            data['token_type_ids'] = data['token_type_ids'].to(device)
            outputs = model(input_ids = data['input_ids'],attention_mask = data['attention_mask']
                            #,token_type_ids = data['token_type_ids']
                            )
            loss = criterion(outputs.logits, labels) 
            
        
            optimizer.zero_grad()

            loss.backward()
            optimizer.step()
            scheduler.step()

            _, pred = torch.max(outputs.logits, 1)
            total += labels.size(0)
            
            correct += (pred == labels).sum().item()
            preds += pred.to("cpu").tolist()
            gt += labels.to("cpu").tolist()
            pbar.set_postfix({"Running accuracy" : correct/total, "Running Loss" : loss.item()})
            running_loss += loss.item()
            run_count +=1
            
                
        

    return model
model = train_loop(model,num_epochs,train_dataloader)

100%|██████████| 195/195 [06:27<00:00,  1.99s/it, Running accuracy=0.988, Running Loss=0.124]


In [12]:
class TestDataset(Dataset):
  def __init__(self,premise,hypothesis,tokenizer):
    premise = self.clean_text(premise)
    hypothesis = self.clean_text(hypothesis)
    self.len=len(premise)
    label2idx={key:value for value,key in enumerate(self.get_label_list())}
    self.encodings = tokenizer(premise,hypothesis, add_special_tokens = True , return_tensors = 'pt', padding = True, truncation = True, max_length = 120)

  def __len__(self):
    return self.len
    
  def __getitem__(self,idx):
    item = {key: val[idx] for key,val in self.encodings.items()}
    return item

  def clean_text(self,series):
    return [self.clean(sent) for sent in series]

  def clean(self,sent):
    sent = re.sub('\u00A0',' ',sent)
    sent = re.sub(' +',' ',sent)
    return sent
  
  def get_label_list(self):
    return ['entailment','neutral','contradiction']

In [27]:
torch.cuda.empty_cache()

In [28]:

test_dataset = TestDataset(test['premise'],test['hypothesis'],tokenizer)


test_dataloader = DataLoader(
    dataset = test_dataset,
    pin_memory = (torch.cuda.is_available()),
    shuffle=False,
    batch_size=1,
    num_workers = 4,
    drop_last = False
)
len(test_dataloader)

1666

In [ ]:
predictions = []
logits_savings = []
count = 0
submission = pd.read_csv(os.path.join(data_path,'sample_submission.csv'))

idx2label = {0:'entailment',1:'neutral',2:'contradiction'}
for idx,data in tqdm(enumerate(test_dataloader)):
    data['attention_mask'] = data['attention_mask'].to(device)
    data['input_ids'] = data['input_ids'].to(device)
    data['token_type_ids'] = data['token_type_ids'].to(device)
    # print(data['input_ids'][0])
    outputs = model(input_ids = data['input_ids'],attention_mask = data['attention_mask']
                    #,token_type_ids = data['token_type_ids']
                    )
    
    pred = torch.argmax(outputs.logits,dim=1)
    pred = [idx2label[i.item()] for i in pred]
    predictions+=pred



print(predictions)


In [ ]:

for idx,data in tqdm(enumerate(test_dataloader)):
    data['attention_mask'] = data['attention_mask']
    data['input_ids'] = data['input_ids']
    data['token_type_ids'] = data['token_type_ids']
    
    outputs = model(input_ids = data['input_ids'],attention_mask = data['attention_mask']
                    #,token_type_ids = data['token_type_ids']
                    )
    logits_savings.append(outputs.logits)

281it [00:53,  5.25it/s]

In [ ]:
len(predictions)

In [1]:
submission["label"] = predictions


NameError: ignored

In [24]:
submission.to_csv("/content/gdrive/MyDrive/Data/submissions/FINETUNE_03.csv", index = False)


In [18]:
_from collections import Counter
Counter(test_frame['label'])

Counter({'contradiction': 513, 'entailment': 519, 'neutral': 634})